<a href="https://colab.research.google.com/github/lohaoxi/basic-pytorch-gans/blob/master/lenet-5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import torch
import torchvision
import numpy as np
from torchvision.datasets import MNIST
import torch.nn as nn
import torch.nn.functional as F
from  torch.utils.data import DataLoader



print(torch.__version__)
print(torchvision.__version__)
print(torch.cuda.is_available())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


1.9.0+cu102
0.10.0+cu102
True


In [ ]:
BATCH_SIZE = 32
train_data = MNIST('./data/mnist', 
                   train = True,
                   download = True,
                   transform = torchvision.transforms.Compose([torchvision.transforms.Pad(2), torchvision.transforms.ToTensor() ]))
test_data = MNIST('./data/mnist', 
                  train = False,
                  download = True,
                  transform = torchvision.transforms.Compose([torchvision.transforms.Pad(2),  torchvision.transforms.ToTensor()]))

train_loader = DataLoader(dataset = train_data,
                          batch_size = BATCH_SIZE,
                          shuffle = True,
                          num_workers = 4)

test_loader = DataLoader(dataset = test_data,
                         batch_size = BATCH_SIZE,
                         shuffle = False,
                         num_workers = 4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
for a,b in train_loader:
    temp = a,b
    break

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
print(temp[0].shape)
print(temp[1].shape)

torch.Size([32, 1, 32, 32])
torch.Size([32])


In [ ]:
def onehot_encoder(trg):
    trg = trg.cpu().numpy()
    trg_size = trg.size
    onehot = np.zeros((trg_size, 10))
    onehot[np.arange(trg_size), trg] = 1
    onehot = torch.from_numpy(onehot).float()
    return onehot

# Cauchy-Schwarz Divergence
class CSD(torch.nn.Module):

    def __init__(self):
        super(CSD, self).__init__()

    def forward(self, outputs, trg):
        y = onehot_encoder(trg).to(device)
        nominator = torch.sum(torch.mm(outputs, y.t()), dim = 1)
        denominator = torch.norm(outputs, 2) * torch.norm(y, 2)
        return torch.mean(-1 * torch.log(nominator / denominator))

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, 
                 conv_in_dim, 
                 conv_out_dim, 
                 conv_krn_size, 
                 conv_stride, 
                 conv_b, 
                 maxpool_krn_size, 
                 max_pool_stride):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels = conv_in_dim,
                              out_channels = conv_out_dim,
                              kernel_size = conv_krn_size,
                              stride = conv_stride,
                              bias = conv_b)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size = maxpool_krn_size,
                                    stride = max_pool_stride)
    def forward(self, batch, pool = True):
        batch = self.conv(batch)
        batch = self.relu(batch)
        if pool == True:
            batch = self.maxpool(batch)
        elif pool == False:
            batch = batch
        return batch

class FeedForward(nn.Module):
    def __init__(self, 
                 layer_1_in_dim,
                 layer_1_out_dim,
                 layer_1_b,
                 layer_2_in_dim,
                 layer_2_out_dim,
                 layer_2_b):
        super(FeedForward, self).__init__()
        self.layer_1 = nn.Linear(in_features = layer_1_in_dim,
                                out_features = layer_1_out_dim,
                                bias = layer_1_b)
        self.layer_2 = nn.Linear(in_features = layer_2_in_dim,
                                out_features = layer_2_out_dim,
                                bias = layer_2_b)
        self.relu = nn.ReLU()
        
    def forward(self, batch):
        batch = self.layer_1(batch)
        batch = self.relu(batch)
        batch = self.layer_2(batch)
        return batch


In [ ]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv_1 = ConvBlock(conv_in_dim = 1,  
                                conv_out_dim = 6, 
                                conv_krn_size = (5, 5), 
                                conv_stride = 1,  
                                conv_b = True,  
                                maxpool_krn_size = (2, 2),  
                                max_pool_stride= 2)
        
        self.conv_2 = ConvBlock(conv_in_dim = 6,  
                                conv_out_dim = 16, 
                                conv_krn_size = (5, 5), 
                                conv_stride = 1,  
                                conv_b = True,  
                                maxpool_krn_size = (2, 2),  
                                max_pool_stride= 2)
        
        self.conv_3 = ConvBlock(conv_in_dim = 16,  
                                conv_out_dim = 120, 
                                conv_krn_size = (5, 5), 
                                conv_stride = 1,  
                                conv_b = True,  
                                maxpool_krn_size = (2, 2),  
                                max_pool_stride= 2)
        
        self.fc = FeedForward(layer_1_in_dim = 120,
                              layer_1_out_dim = 84,
                              layer_1_b = True,
                              layer_2_in_dim = 84,
                              layer_2_out_dim = 10,
                              layer_2_b = True)

    def forward(self, batch):
        batch = self.conv_1(batch, pool = True)
        batch = self.conv_2(batch, pool = True)
        batch = self.conv_3(batch, pool = False)
        batch = batch.view(batch.size(0), -1)
        batch = self.fc(batch)
        batch = F.softmax(batch)
        return batch
                                
        

In [ ]:
model = LeNet5().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)
criterion = CSD()
print(model)

LeNet5(
  (conv_1): ConvBlock(
    (conv): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (relu): ReLU()
    (maxpool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_2): ConvBlock(
    (conv): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (relu): ReLU()
    (maxpool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_3): ConvBlock(
    (conv): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (relu): ReLU()
    (maxpool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): FeedForward(
    (layer_1): Linear(in_features=120, out_features=84, bias=True)
    (layer_2): Linear(in_features=84, out_features=10, bias=True)
    (relu): ReLU()
  )
)


In [ ]:
EPOCHS = 128
steps = len(train_loader) // BATCH_SIZE
start_time = time.time()

model.train(True)

for epoch in range(EPOCHS):
    
    epoch_loss = 0

    performed_steps = 0
    
    for i, (src, trg) in enumerate(train_loader):
        
        if i == steps: break

        src = src.to(device)
        trg = trg.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(src)
        
        loss = criterion(outputs, trg)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss
        
        performed_steps += 1
    
    print("Epoch: {}, Loss: {:0.4f}, Elapsed Time: {:0.6f}".format(epoch + 1, 
                                                                   epoch_loss / steps, 
                                                                   time.time() - start_time)
    )


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1, Loss: 1.1548, Elapsed Time: 0.807044
Epoch: 2, Loss: 1.1511, Elapsed Time: 1.437001
Epoch: 3, Loss: 1.1524, Elapsed Time: 2.060158
Epoch: 4, Loss: 1.1492, Elapsed Time: 2.684847
Epoch: 5, Loss: 1.1502, Elapsed Time: 3.323942
Epoch: 6, Loss: 1.1495, Elapsed Time: 3.940490
Epoch: 7, Loss: 1.1497, Elapsed Time: 4.574377
Epoch: 8, Loss: 1.1498, Elapsed Time: 5.196639
Epoch: 9, Loss: 1.1514, Elapsed Time: 5.816620
Epoch: 10, Loss: 1.1497, Elapsed Time: 6.450045
Epoch: 11, Loss: 1.1506, Elapsed Time: 7.068037
Epoch: 12, Loss: 1.1513, Elapsed Time: 7.679722
Epoch: 13, Loss: 1.1486, Elapsed Time: 8.309180
Epoch: 14, Loss: 1.1497, Elapsed Time: 8.939669
Epoch: 15, Loss: 1.1499, Elapsed Time: 9.560322
Epoch: 16, Loss: 1.1504, Elapsed Time: 10.181568
Epoch: 17, Loss: 1.1489, Elapsed Time: 10.799386
Epoch: 18, Loss: 1.1506, Elapsed Time: 11.419469
Epoch: 19, Loss: 1.1490, Elapsed Time: 12.032374
Epoch: 20, Loss: 1.1487, Elapsed Time: 12.647814
Epoch: 21, Loss: 1.1477, Elapsed Time: 13.30

In [ ]:
avg_loss = 0
avg_acc = 0

model.train(False)

with torch.no_grad():
    
    steps = 0

    for src, trg in test_loader:
        
        src = src.to(device)
        trg = trg.to(device)

        outputs = model(src)

        avg_loss += criterion(outputs, trg)

        _, preds = torch.max(outputs, 1)
        avg_acc += preds.eq(trg).sum().item()
        
        steps += 1

print("Loss: {:0.2f}, Acc: {:.2%}".format(avg_loss / steps,
                                          avg_acc / (steps * BATCH_SIZE))
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Loss: 1.15, Acc: 86.04%
